<a href="https://colab.research.google.com/github/vivekslair/experimentations/blob/main/GL_GrandHackathon_RAPIDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
!nvidia-smi

Tue Dec 22 08:05:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
PLEASE READ
********************************************************************************************************
Changes:
1. IMPORTANT CHANGES: RAPIDS on Colab will be pegged to 0.14 Stable until further notice.
2. Default stable version is now 0.14.  Nightly will redirect to 0.14.
3. You can now declare your RAPIDSAI version as a CLI option and skip the user prompts (ex: '0.14' or '0.15', between 0.13 to 0.14, without the quotes): 
        "!bash rapidsai-csp-utils/colab/rapids-colab.sh <version/label>"
        Examples: '!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.14', or '!bash rapidsai-csp-utils/colab/rapids-colab.sh stable', or '!bash rapidsai-csp-utils/colab/rapids-colab.sh s'
                  '!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.15, or '!bash rapidsai-csp-utils/colab/rapids-colab.sh nightly', or '!bash rapidsai-csp-utils/colab/rapids-colab.sh n'
Enjoy using RAPIDS!  

In [ ]:
!conda install --yes --prefix /usr/local pyarrow pynvml

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done

# All requested packages already installed.



In [ ]:
!cp /usr/local/lib/python3.6/dist-packages/llvmlite/binding/libllvmlite.so .  # to rectify the Error that llvvmlite.so couldnt be loaded

In [ ]:
!conda install -c numba llvmlite # to rectify issue : ImportError: Numba requires at least version 9.0.0 of LLVM.

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done

# All requested packages already installed.



In [ ]:
import cudf # make changes in the _init_.py file in the numba package folder in /usr/local/lib/python3.6/site-packages/numba
import cuml

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.chdir('/content/drive/My Drive/GLHackathonFiles')

In [ ]:
zf = zipfile.ZipFile('Train Data.zip') 
freq_domain_df = cudf.read_csv(zf.open('Train Data Zip/frequency_domain_features_train.csv'))

In [ ]:
heart_rt_df = cudf.read_csv(zf.open('Train Data Zip/heart_rate_non_linear_features_train.csv'))

In [ ]:
time_domain_df = cudf.read_csv(zf.open('Train Data Zip/time_domain_features_train.csv'))

In [ ]:
freq_domain_df.shape

(369289, 12)

In [ ]:
heart_rt_df.shape

(369289, 7)

In [ ]:
time_domain_df.shape

(369289, 20)

In [ ]:
train_df = freq_domain_df.merge(heart_rt_df,on='uuid').merge(time_domain_df,on='uuid')

In [ ]:
train_df = train_df[['uuid', 'VLF', 'VLF_PCT', 'LF', 'LF_PCT', 'LF_NU', 'HF', 'HF_PCT',
       'HF_NU', 'TP', 'LF_HF', 'HF_LF', 'SD1', 'SD2', 'sampen', 'higuci',
       'datasetId', 'condition', 'MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD',
       'SDSD', 'SDRR_RMSSD', 'pNN25', 'pNN50', 'KURT', 'SKEW',
       'MEAN_REL_RR', 'MEDIAN_REL_RR', 'SDRR_REL_RR', 'RMSSD_REL_RR',
       'SDSD_REL_RR', 'SDRR_RMSSD_REL_RR', 'KURT_REL_RR', 'SKEW_REL_RR','HR']]

Converting condition to One hot encoded values

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')

In [ ]:
codes = train_df['condition'].unique()

In [ ]:
train_df = train_df.one_hot_encoding('condition','{}_dummy'.format('condition'),codes)

In [ ]:
train_df = train_df[['uuid','VLF','VLF_PCT','LF','LF_PCT','LF_NU','HF','HF_PCT','HF_NU','TP','LF_HF','HF_LF','SD1','SD2','sampen','higuci','MEAN_RR','MEDIAN_RR','SDRR','RMSSD','SDSD','SDRR_RMSSD','pNN25','pNN50','KURT','SKEW','MEAN_REL_RR','MEDIAN_REL_RR','SDRR_REL_RR','RMSSD_REL_RR','SDSD_REL_RR','SDRR_RMSSD_REL_RR','KURT_REL_RR', 'SKEW_REL_RR','condition_dummy_interruption','condition_dummy_no stress','condition_dummy_time pressure','HR']]

In [ ]:
X=train_df.drop(columns=['HR','uuid'],axis=1)
y=train_df['HR']

In [ ]:
X = X.astype(np.float32)
y = y.astype(np.float32)

In [ ]:
from cuml.preprocessing.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((295432, 36), (73857, 36))

#Random Forest Regression

In [ ]:
from cuml.ensemble import RandomForestRegressor as curfr
from cuml.metrics import accuracy_score

In [ ]:
import cupy as cp

In [ ]:
train_cupy = cp.fromDlpack(X_train.to_dlpack())

/usr/local/lib/python3.6/site-packages/cudf/io/dlpack.py:74: UserWarning: WARNING: cuDF to_dlpack() produces column-major (Fortran order) output. If the output tensor needs to be row major, transpose the output of this function.
  return libdlpack.to_dlpack(gdf_cols)


In [ ]:
train_cupy = train_cupy.astype('float32')

In [ ]:
%%time
cuml_model = curfr(n_estimators=500,
                   max_depth=50,
                   seed=10)

#cuml_model.fit(train_cupy, y_train)

CPU times: user 460 µs, sys: 0 ns, total: 460 µs
Wall time: 446 µs


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Setting the random seed does not fully guarantee the exact same results at this time.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#train_cupy = cp.fromDlpack(X_train.to_dlpack())

In [ ]:
#test_cupy = cp.fromDlpack(X_test.to_dlpack())

In [ ]:
#test_cupy = test_cupy.astype('float32')

In [ ]:
from cuml.metrics.regression import mean_absolute_error

In [ ]:
#y_test_cupy = cp.fromDlpack(y_test.to_dlpack())

In [ ]:
#y_pred_cupy = y_pred.astype('float64')

In [ ]:
cv = KFold(n_splits=7, random_state=42, shuffle=True)
accuracy_model = []
for train_index, test_index in cv.split(X):
    # Split train-test
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Train the model
    model = cuml_model.fit(X_train, y_train)
    # Append to accuracy_model the accuracy of the model
    accuracy_model.append(mean_absolute_error(y_test, model.predict(X_test,predict_model='CPU')))

In [ ]:
regressor.feature_importances_

In [ ]:
train_df_cols = pd.DataFrame(X.columns)

In [ ]:
train_df_cols['weights'] = regressor.feature_importances_

In [ ]:
train_df_cols.rename(columns={0:'column_nm'},inplace=True)

In [ ]:
train_df_cols.sort_values(by='weights',ascending=False,inplace=True)

In [ ]:
plot_data = train_df_cols.iloc[0:5,:]

In [ ]:
sns.barplot(y=plot_data['column_nm'],x=plot_data['weights'])

#Final prediction pipeline

In [ ]:
zf = zipfile.ZipFile('Test Data.zip')

In [ ]:
test_freq = pd.read_csv(zf.open('Test Zip/frequency_domain_features_test.csv'))

In [ ]:
test_heart_rate = pd.read_csv(zf.open('Test Zip/heart_rate_non_linear_features_test.csv'))

In [ ]:
test_time_domain = pd.read_csv(zf.open('Test Zip/time_domain_features_test.csv'))

In [ ]:
test_df = test_freq.merge(test_heart_rate,on='uuid').merge(test_time_domain,on='uuid')

In [ ]:
test_enc_df = pd.DataFrame(enc.transform(test_df[['condition']]).toarray())

In [ ]:
test_df = test_df.join(test_enc_df)

In [ ]:
test_df = test_df.rename(columns={0:'condition_0',1:'condition_1',2:'condition_2'})

In [ ]:
test_df.columns

In [ ]:
test_df.drop(columns=['uuid','condition','datasetId'],axis=1,inplace=True)

In [ ]:
test_df.shape

In [ ]:
test_df.columns

In [ ]:
X_train.columns

In [ ]:
test_pred = regressor.predict(test_df)

In [ ]:
prediction_data = test_freq.merge(test_heart_rate,on='uuid').merge(test_time_domain,on='uuid')
#prediction_data = prediction_data['uuid']

In [ ]:
prediction_data['HR'] = test_pred

In [ ]:
prediction_data.head()

In [ ]:
final_df = prediction_data[['uuid','HR']]

In [ ]:
final_df.to_csv('final_prediction.csv')